## Generate a Complete Database 

- date : 24/05/2024
- New features : generate a complete database using "OrdalieTech/Solon-embeddings-large-0.1" embedding model (config file have been improved)
- Expected improvement : improving the retrieval capabilities with a much strong embedding model.

In [1]:
import sys
import os
import pandas as pd

sys.path.append(os.path.abspath(os.path.join(os.path.dirname(os.path.abspath('')), '..', 'llm-open-data-insee/src')))

In [3]:
from db_building import  build_database_from_csv
db = build_database_from_csv('/home/onyxia/work/llm-open-data-insee/data_complete.csv')

2024-05-24 07:31:45,162 - INFO - The database will be stored in data/chroma_database/chroma_db/
2024-05-24 07:31:45,164 - INFO - The path '/home/onyxia/work/llm-open-data-insee/data_complete.csv' exists.
2024-05-24 07:31:45,165 - INFO - Start building the database
2024-05-24 07:32:07,218 - INFO - Extracting paragraphs and metadata
43226it [00:59, 722.25it/s] 
2024-05-24 07:33:07,247 - INFO - Remove NaN values by empty strings
2024-05-24 07:33:07,282 - INFO - Building the list of Document objects


chunck size :  512
chunck overlap size :  51


Token indices sequence length is longer than the specified maximum sequence length for this model (525 > 512). Running this sequence through the model will result in indexing errors
2024-05-24 07:36:27,580 - INFO - Number of created chuncks : 95249 in the Vector Database
2024-05-24 07:36:27,827 - INFO - Storing the Document objects
2024-05-24 07:36:28,374 - INFO - Load pretrained SentenceTransformer: OrdalieTech/Solon-embeddings-large-0.1
/opt/mamba/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
2024-05-24 07:36:33,165 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2024-05-24 07:59:08,755 - INFO - The database has been built
/opt/mamba/lib/python3.12/site-packages/langchain_core/_api/deprec

In [5]:
db.similarity_search("Quels sont les chiffres du chômages en 2023")

[Document(page_content="L’essentiel sur… le chômage fournit des éléments de cadrage pour aborder ces questions, à l’aide de chiffres clés représentés de manière visuelle et d’un court commentaire. Au quatrième trimestre 2023, le nombre de chômeurs au sens du Bureau international du travail (BIT) augmente de 29000 par rapport au trimestre précédent, à 2,3millions de personnes. Le taux de chômage au sens du BIT s'établit à 7,5% de la population active en France (hors Mayotte), stable par rapport au troisième trimestre 2023 dont l’estimation est très légèrement rehaussée (+0,1 point, de 7,4 % à 7,5 %). Le taux de chômage est supérieur de 0,4 point à son niveau de fin 2022, qui était le plus bas depuis 1982 ; il demeure ainsi nettement au-dessous de son pic de mi-2015 (-3,0 points). Le taux de chômage des jeunes est nettement supérieur à celui du reste de la population. En 2023, il atteint 17,2 % en moyenne sur l'année chez les 15-24 ans, contre 6,7 % chez les 25-49 ans et 5,1 % chez les 5